<a href="https://colab.research.google.com/github/gmkmuralikarthik/Motion-classifier/blob/main/LSTM_motion_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from numpy import mean
from numpy import std
from numpy import dstack
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import LSTM, TimeDistributed, Dense
from keras.utils import to_categorical, plot_model
from matplotlib import pyplot
from keras.utils.np_utils import to_categorical



In [2]:
path = '/content/drive/My Drive/Colab Notebooks/motion_classifier'
dfwalk = pd.read_csv(path + '/walking.csv', header=None)
dfrun = pd.read_csv(path + '/running.csv', header=None)
dfidle = pd.read_csv(path + '/idle.csv', header=None)
columns = ['x','y','z','class']
dfwalk.columns= columns
dfrun.columns= columns
dfidle.columns= columns

In [3]:
dfwalk.loc[dfwalk['class']=='walking','class'] = 1
dfrun.loc[dfrun['class']=='running','class'] = 2
dfidle.loc[dfidle['class']=='idle', 'class'] = 0

In [4]:
dfwalk.isnull().any()

x        False
y        False
z        False
class    False
dtype: bool

In [5]:
dfrun.isnull().any() 

x        False
y        False
z        False
class    False
dtype: bool

In [6]:
nan_rows = dfwalk[dfwalk['x'].isnull()]

In [7]:
nan_rows

,x,y,z,class


In [8]:
dfidle.isnull().any() 

x        False
y        False
z        False
class    False
dtype: bool

In [9]:
train = dfrun.loc[:dfrun.shape[0]*.7]
test = dfrun.loc[dfrun.shape[0]*.7:dfrun.shape[0]]
train = train.append(dfwalk.loc[:dfwalk.shape[0]*.7])
test = test.append(dfwalk.loc[dfwalk.shape[0]*.7:dfwalk.shape[0]])
train = train.append(dfidle.loc[:dfidle.shape[0]*.7])
test = test.append(dfidle.loc[dfidle.shape[0]*.7:dfidle.shape[0]])
# X = dfidle.loc[:]
# X = X.append(dfwalk)
# X = X.append(dfrun)

In [10]:
train.isnull().any() 

x        False
y        False
z        False
class    False
dtype: bool

In [11]:
test.isnull().any() 

x        False
y        False
z        False
class    False
dtype: bool

In [12]:
testY = test['class']
trainY = train['class']
trainX = train[['x','y','z']]
testX = test[['x','y','z']]

In [13]:
testX.shape

(10046, 3)

In [14]:
trainX[:10]

,x,y,z
0,0.677,6.344,8.114
1,0.950,6.248,8.030
2,1.432,6.123,8.403
3,1.475,6.047,8.250
4,1.101,6.174,7.826
5,0.838,6.100,7.498
6,0.739,6.068,7.342
7,0.905,6.020,7.519
8,1.192,6.018,7.366
9,1.247,6.071,7.436


In [15]:
# X = X.to_numpy()
# Y = Y.to_numpy()
testY = testY.to_numpy(testY)
trainY = trainY.to_numpy(trainY)
trainX = trainX.to_numpy()
testX = testX.to_numpy()
testY = np.array(testY,dtype='int64')
trainY = np.array(trainY,dtype='int64')
# Y = np.array(Y,dtype='int64')

In [16]:
testX.shape

(10046, 3)

In [17]:
trainX[:10]

array([[0.677, 6.344, 8.114],
       [0.95 , 6.248, 8.03 ],
       [1.432, 6.123, 8.403],
       [1.475, 6.047, 8.25 ],
       [1.101, 6.174, 7.826],
       [0.838, 6.1  , 7.498],
       [0.739, 6.068, 7.342],
       [0.905, 6.02 , 7.519],
       [1.192, 6.018, 7.366],
       [1.247, 6.071, 7.436]])

In [18]:
# X = X.reshape((int(X.shape[0]/12),12,3))
def process(X,Y):
  test = []
  train = []
  t = 20
  for i in range(t,X.shape[0], t):
    test.append(X[i-t:i, :])
    train.append(np.bincount(Y[i-t:i]).argmax())
    # print(max(np.bincount(Y[i-12:i])))
  X = np.array(test)
  Y = np.array(train,dtype='float64')
  return X, Y
# X.shape


In [19]:
testx, testy = process(testX, testY)
trainx, trainy = process(trainX, trainY)
del trainX
del trainY
del testX
del testY

In [20]:
testy = to_categorical(testy)
trainy = to_categorical(trainy)
# trainx = X[:int(X.shape[0]*.7),:,:]
# testx = X[int(X.shape[0]*.7):,:,:]
# Y = to_categorical(Y)
# trainy = Y[:int(Y.shape[0]*.7)]
# testy = Y[int(Y.shape[0]*.7):]

In [21]:
trainx.shape

(1172, 20, 3)

In [22]:
trainy.shape

(1172, 3)

In [23]:
testx.shape

(502, 20, 3)

In [24]:
testy.shape

(502, 3)

In [25]:
def summarize_results(scores):
	print(scores)
	m, s = mean(scores), std(scores)
	print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

In [26]:
def evaluate_model(trainx, trainy, testx, testy):
	verbose, epochs, batch_size = 0, 5, 16
	n_timesteps, n_features, n_outputs = trainx.shape[1], trainx.shape[2], trainy.shape[1]
	model = Sequential()
	model.add(LSTM(16, input_shape=(n_timesteps,n_features)))
	model.add(Dropout(0.2))
	# model.add(Dense(16, activation='relu'))
	model.add(Dense(n_outputs, activation='softmax'))
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'],)
	model.fit(trainx, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
	loss, accuracy = model.evaluate(testx, testy, batch_size=batch_size, verbose=1)
	return loss, accuracy, model

In [27]:
def run_experiment(repeats=10):
	# load data
	# trainX, trainy, testX, testy = load_dataset()
	# repeat experiment
	scores = list()
	for r in range(repeats):
		loss, score, model = evaluate_model(trainx, trainy, testx, testy)
		score = score * 100.0
		print('>#%d: %.3f' % (r+1, score))
		print('>#%d: %.3f' % (r+1, loss))
		scores.append(score)
		# summarize results
	
	summarize_results(scores)
	return model

In [28]:
model = run_experiment()

32/32 [==============================] - 0s 2ms/step - loss: 0.1874 - accuracy: 0.9482
>#1: 94.821
>#1: 0.187
32/32 [==============================] - 0s 2ms/step - loss: 0.2172 - accuracy: 0.9502
>#2: 95.020
>#2: 0.217
32/32 [==============================] - 0s 2ms/step - loss: 0.2550 - accuracy: 0.9024
>#3: 90.239
>#3: 0.255
32/32 [==============================] - 0s 2ms/step - loss: 0.1677 - accuracy: 0.9442
>#4: 94.422
>#4: 0.168
32/32 [==============================] - 0s 3ms/step - loss: 0.1741 - accuracy: 0.9502
>#5: 95.020
>#5: 0.174
32/32 [==============================] - 0s 2ms/step - loss: 0.1467 - accuracy: 0.9522
>#6: 95.219
>#6: 0.147
32/32 [==============================] - 0s 3ms/step - loss: 0.2231 - accuracy: 0.9382
>#7: 93.825
>#7: 0.223
32/32 [==============================] - 0s 2ms/step - loss: 0.1864 - accuracy: 0.9502
>#8: 95.020
>#8: 0.186
32/32 [==============================] - 0s 2ms/step - loss: 0.2668 - accuracy: 0.9143
>#9: 91.434
>#9: 0.267
32/32 [===

In [29]:
model.summary() 

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_9 (LSTM)                (None, 16)                1280      
_________________________________________________________________
dropout_9 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 3)                 51        
Total params: 1,331
Trainable params: 1,331
Non-trainable params: 0
_________________________________________________________________


In [30]:
model.save('/content/drive/My Drive/Colab Notebooks/LSTM/motion_classifier.h5', include_optimizer=False)

In [31]:
import tensorflow as tf

saved_model_dir = '/content/drive/My Drive/Colab Notebooks/LSTM/motion_classifier.h5'
# Convert the model
converter = tf.lite.TFLiteConverter.from_keras_model(model) # path to the SavedModel directory
tflite_model = converter.convert()

# Save the model.
with open('/content/drive/My Drive/Colab Notebooks/LSTM/MotionClass.tflite', 'wb') as f:
  f.write(tflite_model)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /tmp/tmp7shz33fa/assets


ConverterError: ignored

THE END

In [ ]:
# !pip install tf-nightly
import tensorflow.compat.v2 as tf
tf.enable_v2_behavior()



In [ ]:
run_model = tf.function(lambda x: model(x))
# This is important, let's fix the input size.
BATCH_SIZE = 16
STEPS = 20
INPUT_SIZE = 3
concrete_func = run_model.get_concrete_function(
    tf.TensorSpec([BATCH_SIZE, STEPS, INPUT_SIZE], model.inputs[0].dtype))

# model directory.
MODEL_DIR = "/content/drive/My Drive/Colab Notebooks/LSTM/motion_class"
model.save(MODEL_DIR, save_format="tf", signatures=concrete_func)

converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR)
tflite_model = converter.convert()

In [ ]:
with open('/content/drive/My Drive/Colab Notebooks/LSTM/MotionClass.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
TEST_CASES = 10

# Run the model with TensorFlow Lite
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print(input_details)

for i in range(TEST_CASES):
  expected = model.predict(testx[i:i+1])
  # print(interpreter.get_input_details()[0]["index"])
  interpreter.set_tensor(input_details[0]["index"], np.array(testx[i:i+1, :, :],dtype= np.float32))
  interpreter.invoke()
  result = interpreter.get_tensor(output_details[0]["index"])

  # Assert if the result of TFLite model is consistent with the TF model.
  np.testing.assert_almost_equal(expected, result)
  print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

  # Please note: TfLite fused Lstm kernel is stateful, so we need to reset
  # the states.
  # Clean up internal states.
  interpreter.reset_all_variables()